In [4]:
# -*- coding : utf-8 -*-

# NOTE
# 재귀 html 페이지 처리 순서
# [1] HTML 분석
# [2] 링크 추출
# [3] 각 링크 대상에 다과 같은 처리를 함.
# [4] 파일을 다운받음.
# [5] 파일이 HTML이라면 재귀적으로 [1]로 돌아가 순서를 처음부터 실행.

In [3]:
from bs4 import BeautifulSoup as bs
from urllib.request import *
from urllib.parse import *
import os, time, re

In [5]:
# NOTE : 이미 처리한 파일인지 확인하기 위한 변수
proc_files = {}

In [6]:
# NOTE : HTML 내부에 있는 링크를 추출하는 함수
def enum_links(html, base):
    soup = bs(html, 'lxml')
    link = soup.select('link[rel="stylesheet"]') # NOTE : CSS
    link += soup.select('a[href]') # NOTE : 링크
    result = []
    
    # NOTE : href 속성을 추출, 링크를 절대 경로로 변환
    for rep in link:
        href = a.attrs['href']
        url = urljoin(base, href)
        result.append(url)
    return result

In [14]:
# NOTE : 파일을 다운받고 저장하는 함수
def download_file(url):
    o = urlparse(url)
    savepath = './' + o.netloc + o.path
    if re.search(r"$", savepath): # NOTE : 폴더라면 index.html
        savepath += 'index.html'
    savedir = os.path.dirname(savepath)
    
    # NOTE : 파일이 모두 다운되었는지 확인
    if os.path.exists(savepath): return savepath
    
    # NOTE : 다운로드받을 폴더 생성
    if not os.path.isdir(savedir):
        print('mkdir = ', savedir)
        os.mkdir(savedir)
    
    # NOTE : 파일 다운받기
    try:
        print('download = ', url)
        urlretrieve(url, savepath)
    except:
        prit("다운로드 실패 : ", url)
        return None

In [12]:
# NOTE : HTML을 분석하고 다운받는 함수
def analyze_html(url, root_url):
    savepath = download_file(url)
    if savepath is None : return
    if savepath is proc_files : return # NOTE : 이미 처리됐다면 실행되지 않음.
    proc_files[savepath] = True
    print('analyze_html = ', url)
    
    # NOTE : 링크 추출
    html = open(savepath, "r", encoding = 'utf-8').read()
    links = enum_links(html, url)
    
    for rep in links:
        # NOTE : 링크가 루트 이외의 경로를 나타낸다면 무시
        if rep.find(root_url) != 0:
            if not re.search(r".css$", rep) : continue
        
        # NOTE : HTML이라면
        if re.search(r".(html|htm)$", rep):
            # NOTE : 재귀적으로 HTML 파일 분석
            analyze_html(rep, root_url)
            continue
        # NOTE : 기타 파일
        download_file(rep)
    

In [15]:
# NOTE : URL에 있는 모든것 다운로드
url = "https://docs.python.org/3.5/library"
analyze_html(url, url)

mkdir =  ./docs.python.org/3.5


FileNotFoundError: [Errno 2] No such file or directory: './docs.python.org/3.5'